# Importing libraries

In [1]:
import numpy as np
import pandas as pd

# Importing dataset

In [2]:
dataset = pd.read_csv(r'C:\Users\Admin\Downloads\Telecome_Complaint\Dataset\Complaint_Narrative.csv')
dataset.head()

,Complaint Catagory,clean
0,Internet,comcast cable internet speeds
1,Billing,payment disappear - service got disconnected
2,Internet,speed and service
3,Network,comcast imposed a new usage cap of 300gb that ...
4,Network,comcast not working and no service to boot


In [3]:
dataset.shape

(2223, 2)

# Data cleaning

In [4]:
dataset['Complaint Catagory'].unique() #nominal Data

array(['Internet', 'Billing', 'Network', 'Outages', 'Email', 'Other',
       'Grievance'], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder
categorical = ['Complaint Catagory']
label_encoder = LabelEncoder()
for col in categorical:
    label_encoder.fit(dataset[col])
    dataset[col] = label_encoder.transform(dataset[col])

In [6]:
dataset['Complaint Catagory'].unique()

array([3, 0, 4, 6, 1, 5, 2])

In [7]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [8]:
corpus=[]

for i in range(0, 2223):
  review = re.sub('[^a-zA-Z]', ' ', dataset['clean'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [9]:
corpus

['comcast cabl internet speed',
 'payment disappear servic got disconnect',
 'speed servic',
 'comcast impos new usag cap gb punish stream',
 'comcast not work servic boot',
 'isp charg arbitrari data limit overag fee',
 'throttl servic unreason data cap',
 'comcast refus help troubleshoot correct servic',
 'comcast extend outag',
 'comcast rais price not avail ask',
 'bill servic ask disconnect',
 'yahoo failur restor email search featur',
 'comcast violat open internet rule block hbo go access soni consol',
 'internet speed',
 'internet disconnect everi night',
 'internet complaint',
 'internet avail speed',
 'comcast owe claim need return equip never',
 'horribl internet servic',
 'failur provid servic bill',
 'internet time monopoli',
 'horribl cabl servic custom servic',
 'speed',
 'comcast monopoli bundl practic',
 'comcast',
 'bait switch',
 'comcast custom servic',
 'apart manag exclus contract comcast',
 'unabl reach comcast agent internet bill issu',
 'wifi internet not work 

# Data transformation

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

In [11]:
X = cv.fit_transform(corpus).toarray()
y = dataset['Complaint Catagory'].values

In [12]:
import pickle
# Pickle trained Model for use in Flask App
with open('c1_BoW_Sentiment_Model.pkl', 'wb') as to_write:
    pickle.dump(cv, to_write)

# Dividing dataset into training and test set

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Model fitting (Naive Bayes)

In [14]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [15]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)

accuracy_score(y_test, y_pred)

[[ 99   0   2  10   1   0]
 [  0   1   0   0   2   0]
 [  0   0   9   0   2   0]
 [  9   0   0 124   7   1]
 [  6   0   0   4  80   2]
 [  6   0  24   2  26  28]]


0.7662921348314606

In [16]:
with open('finalized.pickle', 'wb') as to_write:
    pickle.dump(classifier, to_write)